In [2]:
import setup 
setup.init_django()

In [3]:
from market.models import StockQuote

In [15]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [16]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

5861

In [17]:
total = 0
for obj in qs:
    total += obj.close_price

In [18]:
total / qs.count()

Decimal('410.6987832451800034123869647')

In [19]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 1, 'avg_price': Decimal('249.292104238410')}, {'company': 2, 'avg_price': Decimal('194.198671084337')}, {'company': 4, 'avg_price': Decimal('606.357545570095')}]>

In [20]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [21]:
rolling_qs

[<StockQuote: StockQuote object (149470)>,
 <StockQuote: StockQuote object (149471)>,
 <StockQuote: StockQuote object (149472)>,
 <StockQuote: StockQuote object (149473)>,
 <StockQuote: StockQuote object (149474)>]

In [22]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 149470 237.0000 237.0000
2 149471 236.9900 236.9950
3 149472 237.0000 236.9966666666666666666666667
4 149473 237.0000 236.9975
5 149474 237.0200 237.0020


In [23]:
lastest_daily_timestamps = (
    StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date - timedelta(days=40), end_date))
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date')
)

acutal_timestamps = [x['latest_time'] for x in lastest_daily_timestamps]

In [28]:
acutal_timestamps

[datetime.datetime(2024, 11, 4, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 5, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 6, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 7, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 8, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 9, 0, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 11, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 12, 23, 50, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 13, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 14, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 15, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 16, 0, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 18, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 19, 23, 55, tzinfo=datetime.timezone.utc),


In [24]:
qs = StockQuote.objects.filter(
    company__ticker=ticker, 
    time__range=(start_date, end_date),
    time__in=acutal_timestamps
)

In [25]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=frame_start, end=0),
    )
).order_by('-time')

In [26]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

149474 237.0200 240.148000000000 2025-01-11 00:55:00+00:00
149463 237.0100 241.684120000000 2025-01-10 23:55:00+00:00
163792 242.4000 242.960280000000 2025-01-09 00:55:00+00:00
163780 242.1600 243.124280000000 2025-01-08 23:55:00+00:00
163605 242.1500 243.340280000000 2025-01-07 23:55:00+00:00


In [27]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
    ma_5=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=-4, end=0),
    ),
    ma_20=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=-19, end=0),
    )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(149474,
 Decimal('237.0200'),
 Decimal('240.148000000000'),
 Decimal('248.476490000000'))